In [ ]:
# ========== IMPORTS ==========
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing import image
import tensorflow as tf

# ========== DEFINE CLASS NAMES ==========
class_names = {
    0: 'Healthy',
    1: 'Rust',
    2: 'Blight',
    3: 'Leaf Spot Gray'
}

# ========== DATA LOADING FUNCTION ==========
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        try:
            img = load_img(os.path.join(folder, filename), target_size=(150, 150))
            img = img_to_array(img)
            images.append(img)
            labels.append(label)
        except Exception as e:
            print(f"Error loading image {filename}: {e}")
    return np.array(images), np.array(labels)

# ========== LOAD IMAGES ==========
healthy_dir = '/content/drive/MyDrive/training/healthy'
rusty_dir =  '/content/drive/MyDrive/training/rust_'
blight_dir = '/content/drive/MyDrive/training/Blight'
leaf_spot_gray_dir = '/content/drive/MyDrive/training/leaf_spot Gray'

healthy_images, healthy_labels = load_images_from_folder(healthy_dir, 0)
rusty_images, rusty_labels = load_images_from_folder(rusty_dir, 1)
blight_images, blight_labels = load_images_from_folder(blight_dir, 2)
leaf_spot_gray_images, leaf_spot_gray_labels = load_images_from_folder(leaf_spot_gray_dir, 3)

# Combine data
X = np.concatenate((healthy_images, rusty_images, blight_images, leaf_spot_gray_images))
y = np.concatenate((healthy_labels, rusty_labels, blight_labels, leaf_spot_gray_labels))

# Check class distribution
import collections
labels = np.argmax(to_categorical(y, num_classes=4), axis=1)
print(collections.Counter(labels))

# Normalize and encode
X = X / 255.0
y = to_categorical(y, num_classes=4)

# Train/test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# ========== MODEL DEFINITION ==========
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(512, activation='relu'),
    Dense(4, activation='softmax')  # 4 classes
])

model.compile(
    loss='categorical_crossentropy',  
    optimizer=RMSprop(learning_rate=1e-4),
    metrics=['accuracy']
)

# ========== MODEL TRAINING ==========
model.fit(
    X_train, y_train,
    epochs=50,
    validation_data=(X_val, y_val)
)




Counter({np.int64(1): 1717, np.int64(0): 1659, np.int64(2): 1608, np.int64(3): 1542})
Epoch 1/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 347s 2s/step - accuracy: 0.5654 - loss: 0.9886 - val_accuracy: 0.7343 - val_loss: 0.6144
Epoch 2/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 390s 2s/step - accuracy: 0.8268 - loss: 0.3974 - val_accuracy: 0.7642 - val_loss: 0.5277
Epoch 3/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 379s 2s/step - accuracy: 0.8833 - loss: 0.2905 - val_accuracy: 0.7519 - val_loss: 0.6985
Epoch 4/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 400s 2s/step - accuracy: 0.9091 - loss: 0.2327 - val_accuracy: 0.8844 - val_loss: 0.2810
Epoch 5/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 379s 2s/step - accuracy: 0.9258 - loss: 0.1812 - val_accuracy: 0.9418 - val_loss: 0.1692
Epoch 6/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 383s 2s/step - accuracy: 0.9453 - loss: 0.1454 - val_accuracy: 0.7802 - val_loss: 0.5696
Epoch 7/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 378s 2s/step - accuracy: 0.9436 - loss: 0.1621 - val_accuracy: 0.8492 - val_loss: 0.4185
Epoch 8/50


In [7]:
loss, acc = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {acc:.2f}")


41/41 ━━━━━━━━━━━━━━━━━━━━ 27s 631ms/step - accuracy: 0.9579 - loss: 0.3262
Validation Accuracy: 0.96


In [8]:
model.save("net.keras")

In [9]:
# Download the trained model
from google.colab import files
files.download("net.keras")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>